# Лекция №3

## На прошлой лекции

[Познакомились с работой на вычислительном кластере](../lecture_2/lecture_2.ipynb#slurm)

[Познакомились с базовыми примитивами OpenFOAM](../lecture_2/lecture_2.ipynb#Interpolation)

[Узнали, что примитивов даже больше чем нужно](../lecture_2/lecture_2.ipynb#src/OpenFOAM)

[Узнали про класс Time](../lecture_2/lecture_2.ipynb#Time)

[Рассмотрели формат сетки OpenFOAM](../lecture_2/lecture_2.ipynb#mesh)

## План
1. [Сетка в OpenFOAM](#fvMesh)
2. [Поля в OpenFOAM](#Fields)
3. [Paraview](#Paraview)
4. [dynamicRefineFvMesh](#dynamicRefineFvMesh)
5. [functionObjects](#functionObjects)

## Класс fvMesh <a name="fvMesh"></a>

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | www.openfoam.com
     \\/     M anipulation  |
-------------------------------------------------------------------------------
    Copyright (C) 2011-2017 OpenFOAM Foundation
    Copyright (C) 2019 OpenCFD Ltd.
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    laplacianFoam

Group
    grpBasicSolvers

Description
    Laplace equation solver for a scalar quantity.

    \heading Solver details
    The solver is applicable to, e.g. for thermal diffusion in a solid.  The
    equation is given by:

    \f[
        \ddt{T}  = \div \left( D_T \grad T \right)
    \f]

    Where:
    \vartable
        T     | Scalar field which is solved for, e.g. temperature
        D_T   | Diffusion coefficient
    \endvartable

    \heading Required fields
    \plaintable
        T     | Scalar field which is solved for, e.g. temperature
    \endplaintable

\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "fvOptions.H"
#include "simpleControl.H"

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
    argList::addNote
    (
        "Laplace equation solver for a scalar quantity."
    );

    #include "postProcess.H"

    #include "addCheckCaseOptions.H"
    #include "setRootCaseLists.H"
    #include "createTime.H"
    #include "createMesh.H"

    simpleControl simple(mesh);

    #include "createFields.H"

    // * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

    Info<< "\nCalculating temperature distribution\n" << endl;

    while (simple.loop())
    {
        Info<< "Time = " << runTime.timeName() << nl << endl;

        while (simple.correctNonOrthogonal())
        {
            fvScalarMatrix TEqn
            (
                fvm::ddt(T) - fvm::laplacian(DT, T)
             ==
                fvOptions(T)
            );

            fvOptions.constrain(TEqn);
            TEqn.solve();
            fvOptions.correct(T);
        }

        #include "write.H"

        runTime.printExecutionTime(Info);
    }

    Info<< "End\n" << endl;

    return 0;
}
```

In [ ]:
#!cat $WM_PROJECT_DIR/src/OpenFOAM/include/createMesh.H

```c++
Foam::autoPtr<Foam::fvMesh> meshPtr(nullptr);
Foam::word regionName = Foam::fvMesh::defaultRegion;

if (args.found("dry-run") || args.found("dry-run-write"))
{
    Foam::Info
        << "Operating in 'dry-run' mode: case will run for 1 time step.  "
        << "All checks assumed OK on a clean exit" << Foam::endl;

    // Allow region in combination with dry-run
    args.readIfPresent("region", regionName);

    Foam::FieldBase::allowConstructFromLargerSize = true;

    // Create a simplified 1D mesh and attempt to re-create boundary conditions
    meshPtr.reset
    (
        new Foam::simplifiedMeshes::columnFvMesh(runTime, regionName)
    );

    // Stop after 1 iteration of the simplified mesh

    if (args.found("dry-run-write"))
    {
        // Using saWriteNow triggers function objects execute(), write()
        runTime.stopAt(Foam::Time::saWriteNow);

        // Make sure mesh gets output to the current time (since instance
        // no longer constant)
        meshPtr().setInstance(runTime.timeName());
    }
    else
    {
        // Using saNoWriteNow triggers function objects execute(),
        // but not write()
        runTime.stopAt(Foam::Time::saNoWriteNow);
    }

    Foam::functionObject::outputPrefix = "postProcessing-dry-run";
}
else
{
    if (args.readIfPresent("region", regionName))
    {
        Foam::Info
            << "Create mesh " << regionName << " for time = "
            << runTime.timeName() << Foam::nl << Foam::endl;
    }
    else
    {
        Foam::Info
            << "Create mesh for time = "
            << runTime.timeName() << Foam::nl << Foam::endl;
    }

    meshPtr.reset
    (
        new Foam::fvMesh
        (
            Foam::IOobject
            (
                regionName,
                runTime.timeName(),
                runTime,
                Foam::IOobject::MUST_READ
            ),
            false
        )
    );
    meshPtr().init(true);   // initialise all (lower levels and current)
}

Foam::fvMesh& mesh = meshPtr();
```

[fvMesh](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1fvMesh.html) - doxygen (обратить внимание на primitiveMesh и polyMesh)

In [ ]:
#!cat fvMesh/fvMesh_OF.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | Copyright (C) 2011-2015 OpenFOAM Foundation
     \\/     M anipulation  |
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

\*---------------------------------------------------------------------------*/

#include "fvCFD.H"

int main(int argc, char *argv[])
{
#include "setRootCase.H" // args
#include "createTime.H" // runTime
#include "createMesh.H" // mesh
  
  // mesh.C() - массив цетров (вектор) ячеек
  // mesh.Cf() - массив цетров (вектор) граней
  Info << "Hello there, the most recent time folder found is " << runTime.timeName() << nl
       << "The mesh has " << mesh.C().size() << " cells and " << mesh.Cf().size()
       << " internal faces in it. Wubalubadubdub!" << nl << endl;
  
  // It's possible to iterate over every cell in a standard C++ for loop
  for (label cellI = 0; cellI < mesh.C().size(); cellI++)
    if (cellI%20 == 0) // only show every twentieth cell not to spam the screen too much
      Info << "Cell " << cellI << " with centre at " << mesh.C()[cellI] << endl;
  Info << endl; // spacer

  // Each cell is constructed of faces - these may either be internal or constitute a
  // boundary, or a patch in OpenFOAM terms; internal faces have an owner cell
  // and a neighbour.

  for (label faceI = 0; faceI < mesh.owner().size(); faceI++)
    if (faceI%40 == 0)
      Info << "Internal face " << faceI << " with centre at " << mesh.Cf()[faceI]
	   << " with owner cell " << mesh.owner()[faceI]
	   << " and neighbour " << mesh.neighbour()[faceI] << endl;
  Info << endl;

  // Boundary conditions may be accessed through the boundaryMesh object.
  // In reality, each boundary face is also included in the constant/polyMesh/faces
  // description. But, in that file, the internal faces are defined first.
  // In addition, the constant/polyMesh/boundary file defines the starting faceI
  // indices from which boundary face definitions start.
  // OpenFOAM also provides a macro definition for for loops over all entries
  // in a field or a list, which saves up on the amount of typing.
  forAll(mesh.boundaryMesh(), patchI)
    Info << "Patch " << patchI << ": " << mesh.boundary()[patchI].name() << " with "
	 << mesh.boundary()[patchI].Cf().size() << " faces. Starts at total face "
	 << mesh.boundary()[patchI].start() << endl;
  Info << endl;
  
  // Faces adjacent to boundaries may be accessed as follows.
  // Also, a useful thing to know about a face is its normal vector and face area.
  label patchFaceI(0);
  forAll(mesh.boundaryMesh(), patchI)
    Info << "Patch " << patchI << " has its face " << patchFaceI << " adjacent to cell "
	 << mesh.boundary()[patchI].patch().faceCells()[patchFaceI]
	 << ". It has normal vector " << mesh.boundary()[patchI].Sf()[patchFaceI]
	 << " and surface area " << mag(mesh.boundary()[patchI].Sf()[patchFaceI])
	 << endl;
  Info << endl;
  
  // For internal faces, method .Sf() can be called directly on the mesh instance.
  // Moreover, there is a shorthand method .magSf() which returns the surface area
  // as a scalar.
  // For internal faces, the normal vector points from the owner to the neighbour
  // and the owner has a smaller cellI index than the neighbour. For boundary faces,
  // the normals always point outside of the domain (they have "imaginary" neighbours
  // which do not exist).
  
  // It is possible to look at the points making up each face in more detail.
  // First, we define a few shorthands by getting references to the respective
  // objects in the mesh. These are defined as constants since we do not aim to
  // alter the mesh in any way.
  // NOTE: these lists refer to the physical definition of the mesh and thus
  // include boundary faces. Use can be made of the mesh.boundary()[patchI].Cf().size()
  // and mesh.boundary()[patchI].start() methods to check whether the face is internal
  // or lies on a boundary.

  const faceList& fcs = mesh.faces();
  const List<point>& pts = mesh.points();
  const List<point>& cents = mesh.faceCentres();
  
  forAll(fcs,faceI)
    if (faceI%80==0)
      {
	if (faceI<mesh.Cf().size())
	  Info << "Internal face ";
	else
	  {
	    forAll(mesh.boundary(),patchI)
	      if ((mesh.boundary()[patchI].start()<= faceI) &&
		  (faceI < mesh.boundary()[patchI].start()+mesh.boundary()[patchI].Cf().size()))
		{
		  Info << "Face on patch " << patchI << ", faceI ";
		  break; // exit the forAll loop prematurely
		}
	  }
	
	Info << faceI << " with centre at " << cents[faceI]
	     << " has " << fcs[faceI].size() << " vertices:";
	forAll(fcs[faceI],vertexI)
	  // Note how fcs[faceI] holds the indices of points whose coordinates
	  // are stored in the pts list.
	  Info << " " << pts[fcs[faceI][vertexI]];
	Info << endl;
      }
  Info << endl;
  
  // In the original cavity tutorial, on which the test case is based,
  // the frontAndBack boundary is defined as and "empty" type. This is a special
  // BC case which may cause unexpected behaviour as its .Cf() field has size of 0.
  // Type of a patch may be checked to avoid running into this problem if there
  // is a substantial risk that an empty patch type will appear

  label patchID(0);
  const polyPatch& pp = mesh.boundaryMesh()[patchID];
  if (isA<emptyPolyPatch>(pp))
    {
      // patch patchID is of type "empty".
      Info << "You will not see this." << endl;
    }
  
  // Patches may also be retrieved from the mesh using their name. This could be
  // useful if the user were to refer to a particular patch from a dictionary
  // (like when you do when calculating forces on a particular patch).

  word patchName("movingWall");
  patchID = mesh.boundaryMesh().findPatchID(patchName);

  Info << "Retrieved patch " << patchName << " at index " << patchID << " using its name only." << nl << endl;
  
  Info << "Bounding box "<< mesh.bounds() << endl;
  
  point A (0.09, 0.09, 0.009);
  
  point B (0.09, 0.09, 0.09);
  
  Info << "Nearest cell for point "<< A << "is "<< mesh.findCell(A) << endl;
  
  Info << "Nearest cell for point "<< B << "is "<< mesh.findCell(B) << endl;
     
  Info << "Volume for cell " << mesh.findCell(A) << "is " << mesh.V()[mesh.findCell(A)] << endl;
  
  Info << "geometric directions: " << mesh.geometricD() << endl;
  
  Info << "nPoints = " << mesh.nPoints() << endl;
  
  Info << "nInternalPoints = " << mesh.nInternalPoints() << endl;
  
  Info<< "End\n" << endl;
  
  return 0;
}


// ************************************************************************* //
```

In [ ]:
!wclean fvMesh/
!wmake fvMesh/

<img src="fvMesh/mesh.png" width="500">

In [ ]:
!fvMesh_OF -case fvMesh/test/

## Поля в OpenFOAM <a name="Fields"></a>

In [ ]:
#!cat $WM_PROJECT_DIR/applications/solvers/basic/laplacianFoam/laplacianFoam.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | www.openfoam.com
     \\/     M anipulation  |
-------------------------------------------------------------------------------
    Copyright (C) 2011-2017 OpenFOAM Foundation
    Copyright (C) 2019 OpenCFD Ltd.
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    laplacianFoam

Group
    grpBasicSolvers

Description
    Laplace equation solver for a scalar quantity.

    \heading Solver details
    The solver is applicable to, e.g. for thermal diffusion in a solid.  The
    equation is given by:

    \f[
        \ddt{T}  = \div \left( D_T \grad T \right)
    \f]

    Where:
    \vartable
        T     | Scalar field which is solved for, e.g. temperature
        D_T   | Diffusion coefficient
    \endvartable

    \heading Required fields
    \plaintable
        T     | Scalar field which is solved for, e.g. temperature
    \endplaintable

\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "fvOptions.H"
#include "simpleControl.H"

// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
    argList::addNote
    (
        "Laplace equation solver for a scalar quantity."
    );

    #include "postProcess.H"

    #include "addCheckCaseOptions.H"
    #include "setRootCaseLists.H"
    #include "createTime.H"
    #include "createMesh.H"

    simpleControl simple(mesh);

    #include "createFields.H"

    // * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

    Info<< "\nCalculating temperature distribution\n" << endl;

    while (simple.loop())
    {
        Info<< "Time = " << runTime.timeName() << nl << endl;

        while (simple.correctNonOrthogonal())
        {
            fvScalarMatrix TEqn
            (
                fvm::ddt(T) - fvm::laplacian(DT, T)
             ==
                fvOptions(T)
            );

            fvOptions.constrain(TEqn);
            TEqn.solve();
            fvOptions.correct(T);
        }

        #include "write.H"

        runTime.printExecutionTime(Info);
    }

    Info<< "End\n" << endl;

    return 0;
}


// ************************************************************************* //
```

In [ ]:
#!cat $WM_PROJECT_DIR/applications/solvers/basic/laplacianFoam/createFields.H

```c++
Info<< "Reading field T\n" << endl;

volScalarField T
(
    IOobject
    (
        "T",
        runTime.timeName(),
        mesh,
        IOobject::MUST_READ,
        IOobject::AUTO_WRITE
    ),
    mesh
);


Info<< "Reading diffusivity DT\n" << endl;

volScalarField DT
(
    IOobject
    (
        "DT",
        runTime.timeName(),
        mesh,
        IOobject::READ_IF_PRESENT,
        IOobject::AUTO_WRITE
    ),
    mesh,
    dimensionedScalar(dimViscosity, Zero)
);

if (!DT.headerOk())
{
    IOdictionary transportProperties
    (
        IOobject
        (
            "transportProperties",
            runTime.constant(),
            mesh,
            IOobject::MUST_READ_IF_MODIFIED,
            IOobject::NO_WRITE
        )
    );
    DT = dimensionedScalar("DT", dimViscosity, transportProperties);
}

#include "createFvOptions.H"
```

## Field

[Doxygen Field](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1Field.html)

Field<font color='green'>\<Type\></font> - шаблонный класс (<font color='green'>\<Type\></font> - scalar, vector ...)
    
List<font color='green'>\<Type\></font> + map, component, *= ... - массив величин + операции
    
<font color='red'>Нет привязки к сетке!</font>
    
В общем виде реализованы методы маппирования

## DimensionedField

[Doxygen DimensionedField](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1DimensionedField.html)

Field<font color='green'>\<Type\></font> + dimensioned + GeoMesh + regIOobject

Размерность

Привязка к сетке (соответствие между кол-вом элементов в сетке и поле)

Регистр объектов через IOobject

## GeometricField

[Doxygen GeometricField](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1GeometricField.html)

[Field ProgrammerGuide](../books/OFpg.pdf) p.30

Класс самого высокого уровня

Поля внутри сетки + на границе

В памяти хранятся поля на текущем и предыдущем шагах по времени

volField<font color='green'>\<Type\></font> (volScalarField, volVectorField) - поле в ячейках + поле на границе. Основной тип.

surfaceField<font color='green'>\<Type\></font> (surfaceScalarField, surfaceVectorField) - поле на гранях + поле на границе. Вспомогательные поля в операторах дискритизации по пространству.

pointField<font color='green'>\<Type\></font> (pointScalarField, pointVectorField) - поле в узлах + поле на границе

In [ ]:
#!cat Field/Field_OF.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | Copyright (C) 2011-2016 OpenFOAM Foundation
     \\/     M anipulation  |
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    class Field OF example


\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
// #include "dynamicFvMesh.H"
#include "IFstream.H"
#include "OFstream.H"
#include "IOmanip.H"
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
  argList::addOption
    (
        "k",
	"scalar",
        "frequency"
    );

#include "setRootCase.H"
#include "createTime.H"
#include "createMesh.H"

   scalar k = 1; 

 if (args.readIfPresent("k", k))
    {
       Info << "k = " << k << endl;
    }

 Info << "object names: " << mesh.names() << endl;
 
 volScalarField wave                         // vol - поле в ячейках Scalar - скалярное
    (
     IOobject
     (
      "wave",                                // название объекта в регистре
      runTime.timeName(),                    // время
      mesh,                                  // регистр объектов
      IOobject::NO_READ,                     // опция чтения
      IOobject::AUTO_WRITE                   // опция записи
      ),
     mesh,                                   // сетка 
     dimensionedScalar("wave", dimless, 0.0) // начальное размерное значение
     );

 Info << "object names: " << mesh.names() << endl;
 // volScalarField& wave_ = runTime.lookupObjectRef<volScalarField>("wave");
  
  scalar L = Foam::max(mesh.C().component(0)().primitiveField()) - Foam::min(mesh.C().component(0)().primitiveField());
  Info << "L: "<< L << endl;
  
  while(runTime.loop()) // цикл по времени
    {
      forAll(wave, cellI) // цикл по центрам ячеек
	{
	  // wave.wave.primitiveField() - поле в центрах ячеек (const)
	  // wave.wave.primitiveFieldRef() - поле в центрах ячеек
	   wave.primitiveFieldRef()[cellI] = Foam::sin(runTime.value()+(mesh.C()[cellI].x()/L)*3.1415926*k);

	}
      
      // wave.boundaryField() - поле на границе (патчах)
      // wave.oldTime() - поле с предыдущего шага по времени
      // wave.prevIter() - поле с предыдущей итерации
      // wave.correctBoundaryConditions() - обновление граничных условий
      // wave.relax(alpha) - релаксация поля с коэффициентом alpha
      // wave_n+1 = (1-alpha)*wave_n + alpha*wave_n+1

      // wave.read()
      // wave.write()
      // wave.close()
      // wave.store()
      
      runTime.write();
    }
    return 0;
}


// ************************************************************************* //
```

In [ ]:
# прописывае в переменное окружение пути к папкам для записи исполняемых файлов и библиотек
import os
os.environ['FOAM_USER_APPBIN'] = os.environ['FOAM_APPBIN'].replace("user1", os.environ['USER'])
#os.environ['FOAM_USER_APPBIN']
os.environ['PATH']=os.environ['PATH']+":"+os.environ['FOAM_USER_APPBIN']
#os.environ['PATH']
os.environ['FOAM_USER_LIBBIN'] = os.environ['FOAM_LIBBIN'].replace("user1", os.environ['USER'])
#os.environ['FOAM_USER_LIBBIN']

In [ ]:
!wclean Field/
!wmake Field/

In [ ]:
!cat Field/test/system/blockMeshDict

In [ ]:
!cat Field/test/system/controlDict

In [ ]:
!rm -r Field/test/1*
!rm -r Field/test/0.*
!rm -r Field/test/2*
!rm -r Field/test/3*

In [ ]:
!Field_OF -case Field/test/ -k 5

In [ ]:
ls Field/test

In [ ]:
ls Field/test/1

In [ ]:
#!cat Field/test/1/wave

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       volScalarField;
    location    "1";
    object      wave;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dimensions      [0 0 0 0 0 0 0];


internalField   nonuniform List<scalar> 
100
(
-1.0718e-07
...
-2.14359e-07
)
;

boundaryField
{
    movingWall
    {
        type            calculated;
        value           uniform 0;
    }
    fixedWalls
    {
        type            calculated;
        value           uniform 0;
    }
    frontAndBack
    {
        type            empty;
    }
}


// ************************************************************************* //
```

### Результаты

Сетка
<img src="img/mesh.png" width="800" height="200">

Поле wave
<img src="img/field.png" width="800" height="200">

График
<img src="img/plot.png" width="800" height="200">

### *для самостоятельной работы

Инженерные методы для расчета распределения давления на летательных аппаратах

[Anderson](../books/anderson_hyp.pdf) p.51

## Paraview <a name="Paraview"></a>

[Paraview](https://www.paraview.org/) - постпроцессинг, визуализация, обработка данных

[WD_paraview](../books/wd_paraview.pdf) - презентация WolfDynamics

[Данные для презентации](https://disk.yandex.ru/d/u2x8MgJ2aTwnQw) ВСТАВИТЬ ССЫЛКУ! (внимание! ~ 500 Mb)

pvserver - для удаленного подключения к серверу

##  dynamicRefineFvMesh <a name="dynamicRefineFvMesh"></a>

[Doxygen dynamicFvMesh](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1dynamicFvMesh.html)

1. dynamicRefineFvMesh - дробление гексаэдральных ячеек (на 8 частей). (AMR Adaptive Mesh Refinement)
2. dynamicMotionSolverFvMesh - Изменение положения узлов с сохранением топологии (изменяются только координаты в points).
3. dynamicOversetFvMesh - перекрывающиеся (химера, overset) сетки.


In [ ]:
#!cat dynamicRefineMesh/Field_OF_dynamic.C

```c++
/*---------------------------------------------------------------------------*\
  =========                 |
  \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox
   \\    /   O peration     |
    \\  /    A nd           | Copyright (C) 2011-2016 OpenFOAM Foundation
     \\/     M anipulation  |
-------------------------------------------------------------------------------
License
    This file is part of OpenFOAM.

    OpenFOAM is free software: you can redistribute it and/or modify it
    under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    OpenFOAM is distributed in the hope that it will be useful, but WITHOUT
    ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or
    FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License
    for more details.

    You should have received a copy of the GNU General Public License
    along with OpenFOAM.  If not, see <http://www.gnu.org/licenses/>.

Application
    class Field OF example


\*---------------------------------------------------------------------------*/

#include "fvCFD.H"
#include "dynamicRefineFvMesh.H" ////////////////////////////////
#include "IFstream.H"
#include "OFstream.H"
#include "IOmanip.H"
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

int main(int argc, char *argv[])
{
  argList::addOption
    (
        "k",
	"scalar",
        "frequency"
    );

#include "setRootCase.H"
#include "createTime.H"
#include "createDynamicFvMesh.H"  ///////////////////////////////////

   scalar k = 1; 

 if (args.readIfPresent("k", k))
    {
       Info << "k = " << k << endl;
    }
 
  volScalarField wave
    (
     IOobject
     (
      "wave",
      runTime.timeName(),
      mesh,
      IOobject::NO_READ,
      IOobject::AUTO_WRITE
      ),
     mesh,
     dimensionedScalar("wave", dimless, 0.0)
     );
  
  scalar L = mesh.bounds().span().x();
  Info << "L: "<< L << endl;
  
  while(runTime.loop())
    {
      const volScalarField& x =  mesh.C().component(0);
  
      forAll(wave, cellI)
	{
	  wave.primitiveFieldRef()[cellI] = Foam::sin((runTime.value()+mesh.C()[cellI].x()/L)*3.1415926*k);

	}
      
      mesh.update(); ////////////////////////////

      runTime.write();
    }
    return 0;
}


// ************************************************************************* //
```

In [ ]:
!wclean dynamicRefineMesh/
!wmake dynamicRefineMesh/

In [ ]:
ls dynamicRefineMesh/test/constant/

In [ ]:
#!cat dynamicRefineMesh/test/constant/dynamicMeshDict

```c++
/*--------------------------------*- C++ -*----------------------------------*\
| =========                 |                                                 |
| \\      /  F ield         | OpenFOAM: The Open Source CFD Toolbox           |
|  \\    /   O peration     | Version:  v2012                                 |
|   \\  /    A nd           | Website:  www.openfoam.com                      |
|    \\/     M anipulation  |                                                 |
\*---------------------------------------------------------------------------*/
FoamFile
{
    version     2.0;
    format      ascii;
    class       dictionary;
    location    "constant";
    object      dynamicMeshDict;
}
// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //

dynamicFvMesh   dynamicRefineFvMesh;

dynamicRefineFvMeshCoeffs
{
    // How often to refine
    refineInterval  1;
    // Field to base refinement on
    field           wave;
    // Refine field inbetween lower..upper
    lowerRefineLevel 0.5;
    upperRefineLevel 1.0;
    // Have slower than 2:1 refinement
    nBufferLayers   1;
    // Refine cells only up to maxRefinement levels
    maxRefinement   4;
    // Stop refinement if maxCells reached
    maxCells        2000000;
    // Flux field and corresponding velocity field. Fluxes on changed
    // faces get recalculated by interpolating the velocity. Use 'none'
    // on surfaceScalarFields that do not need to be reinterpolated.
    correctFluxes
    (
     (wave none)
    );
    // Write the refinement level as a volScalarField
    dumpLevel       true;
}


// ************************************************************************* //
```

In [ ]:
!Field_OF_dynamic -case dynamicRefineMesh/test/ -k 5

### Результаты

Сетка
<img src="img/mesh_refine.png" width="800" height="200">

Поле wave
<img src="img/field_refine.png" width="800" height="200">

График 1
<img src="img/plot_refine.png" width="800" height="200">

График 2
<img src="img/plot_refine_line_.png" width="800" height="200">

In [ ]:
!grep -ri dynamicRefineFvMesh $WM_PROJECT_DIR/tutorials/

In [ ]:
!refineMesh -help

In [ ]:
!cat $WM_PROJECT_DIR/applications/utilities/mesh/manipulation/refineMesh/refineMesh.C

In [ ]:
!find $WM_PROJECT_DIR -name refineMeshDict

In [ ]:
!cat $WM_PROJECT_DIR/etc/caseDicts/annotated/refineMeshDict

Youtube

[FireFOAM - OpenFOAM - Adaptive Mesh Refinement (AMR)](https://www.youtube.com/watch?v=8V9Xj_fBgJc)

[Kelvin–Helmholtz instability simulation with adaptive mesh refinement](https://www.youtube.com/watch?v=QKuBtPQ6TMs)

[Kelvin-Helmholtz instability simulation with openFOAM](https://www.youtube.com/watch?v=Bs-l9Ur8TnQ)

[Adaptive Mesh Refinement @ Kármán Vortex Street ∇ Holzmann CFD](https://www.youtube.com/watch?v=u-VV3euIsXo)

[OpenFOAM simulation of overflowing with VOF & AMR](https://www.youtube.com/watch?v=U5InaZdgQDw)

##  functionObjects <a name="functionObjects"></a>

Функции для обработки, которые должны вызываться редко или вообще чаще всего не используются (только постобработка, отдельно)

functions в controlDict!

[Doxygen OpenFOAM functionObjects](https://www.openfoam.com/documentation/guides/latest/doc/guide-function-objects.html)

[Time loop](https://www.openfoam.com/documentation/guides/latest/api/classFoam_1_1Time.html#adf77b77054c2d4c23b70556fe767f23e)

чтобы сделаеть свое - гуглить: function objects openfoam chalmers



## Пример использования

In [ ]:
!postProcess -help

In [ ]:
# !postProcess -case Field/test/ -func mag(wave)
# or
# !postProcess -case Field/test/ -func mag\(wave\) -fields (wave)
!postProcess -case Field/test/ -func mag\(wave\) -latestTime

In [ ]:
!ls Field/test/3

чтобы не перекомпилировать солвер каждый раз, когда сделали новую библиотеку - используются динамические библиотеки, которые так же подключаются в controlDict

[инициализация в конструкторе класса Time (libs)](https://www.openfoam.com/documentation/guides/latest/api/Time_8C_source.html#l00879)

пример в controlDict прописываем:

libs (myLib.so) // важно, чтобы библиотека была в переменном окружение (самое простое в FOAM_LIBBIN, FOAM_USER_LIBBIN или LD_LIBRARY_PATH)

## Заключение <a name="finish"></a>

1. [Познакомились с полями в OpenFOAM](#Fields)
2. [Основы Paraview](#Paraview)
3. [Узнали про dynamicRefineFvMesh](#dynamicRefineFvMesh)
4. [Познакомились с functionObjects](#functionObjects)

## В следующей серии ...

Метод контрольного объема, дискретизация по времени, источники!

<table><tr>
<td> <img src="img/ostr.jpeg" width="300" > </td>
<td> <img src="img/time.jpeg" width="400" > </td>
<td> <img src="img/source.jpeg" width="400" > </td>
</tr></table>
